# 携程旅行评论爬虫

导入必要的包并启动自动化浏览器实例

In [10]:
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument("disable-blink-features=AutomationControlled")
browser = webdriver.Chrome(options=options)

## 登录

如果没登录过请登录并保存登录数据

如果保存过登录数据请加载登陆数据

每次启动浏览器实例都是全新的浏览器（不包含任何数据）

In [11]:
browser.get("https://passport.ctrip.com/user/login")

In [7]:
import json

# 保存登录数据
with open('./cookies.json', 'w', encoding='utf-8') as fp:
    fp.write(json.dumps(browser.get_cookies()))

In [12]:
import json

# 应用登录数据
browser.delete_all_cookies()
with open('./cookies.json', 'r', encoding='utf-8') as fp:
    listCookies = json.loads(fp.read())
for cookie in listCookies:
    browser.add_cookie(cookie)

In [13]:
# 刷新浏览器
browser.refresh()

In [5]:
# 上海艺龙酒店
browser.get("https://hotels.ctrip.com/hotels/72966849.html")

In [14]:
# 飘HOME连锁酒店(北京国贸SKP店)
browser.get("https://hotels.ctrip.com/hotels/detail/?hotelId=482534")

尝试读取一页评论

In [15]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(browser.page_source, 'html.parser')
comments = soup.find_all('div', class_ = 'm-reviewCard-item')
for comment in comments:
    print(comment.find('strong').text)
    print(comment.find('div', class_ = 'm-score_appraise').text)
    print(comment.find('div', class_ = 'comment').text)
    rv_date_loc = comment.find('div', class_ = 'reviewDate').text
    rv_date ,rv_loc = rv_date_loc.split(' ')
    rv_loc = rv_loc.replace('发布于','')
    print(rv_date ,rv_loc)
    print('-' * 100)

5
超棒
价格非常合适了一下子的假期和朋友一起来跨年无论是检查的服务还是酒店的卫生程度都很好前端消息就主动帮我们升级了放心每次出门路过前台都是先给我们打招呼，位置非常方便出门就有公交车
2024年1月2日 北京
----------------------------------------------------------------------------------------------------
5
超棒
房间干净！服务人员做卫生很认真。
设施：住宿这里很接地气。跟住在家里一样。
环境：周边环境，离地铁很近。公交出门就是。服务：酒店服务越来越好。前台很友善。很有亲和力！
欢迎礼包:有好多零食可以吃。不担心晚上没吃的。
2023年12月14日 北京
----------------------------------------------------------------------------------------------------
5
超棒
设施：齐全
卫生：房间内干净整洁
环境：房间小，但是整体舒适度还可以。
服务：服务很好，有问必答。

卫生间淋浴水流充足温度可以

商务：出差来北京培训，北京市区内这样的价位住宿性价比算高的。
2023年12月1日 北京
----------------------------------------------------------------------------------------------------
5
超棒
来北京玩在这家酒店住了三天。酒店离大望路地铁站不远，大概一刻钟的步行距离。从酒店到天安门坐地铁大概四五十分钟左右，交通便利。
酒店的客房设计合理，虽然不大但是空间利用得当。房间的床很软睡得非常舒服，客房内每天的欢迎礼包也令人暖心。
工作人员都不仅认真负责，还很热心。下次来北京还会考虑这家酒店。
2023年11月14日 北京
----------------------------------------------------------------------------------------------------
4

在北京旅游，选择这家酒店整体来说价格相对实惠，居住的环境尚可，就是看到水龙头后面的胶都发黑了，我顺手用牙刷刷了一下，全出来了，我心想搞客房的会不会擦掉

翻到下一页的按钮

In [26]:
# 下一页

browser.find_element('xpath', '//*[@id="ibu_hotel_review"]/div/ul[2]/li/div/div[2]/div[2]/div[12]/ul/li[3]/a/i').click()

全自动爬取当前显示酒店的所有评论

In [44]:
import csv
import time
fp = open('comments.csv', 'w', encoding='utf-8')
fcsv = csv.writer(fp)
fcsv.writerow(['评分', '评价', '评论', '日期', '位置'])

# 返回第一页
browser.refresh()

try:
    while True:
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        comments = soup.find_all('div', class_='m-reviewCard-item')
        
        page = soup.find('div', class_='m_num m_num_checked').text
        
        print('正在爬取第{}页'.format(page))
        
        for index,comment in enumerate(comments):
            print('正在爬取第{}页第{}条评论'.format(page, index+1))
            
            score = comment.find('strong').text
            appraise = comment.find('div', class_='m-score_appraise').text
            comment_text = comment.find('div', class_='comment').text
            rv_date_loc = comment.find('div', class_='reviewDate').text
            try:
                rv_date, rv_loc = rv_date_loc.split(' ')
                rv_loc = rv_loc.replace('发布于', '')
            except:
                rv_date = rv_date_loc.replace('发布', '')
                rv_loc = ""
            fcsv.writerow([score, appraise, comment_text, rv_date, rv_loc])
            
        browser.find_element('xpath', '//*[@id="ibu_hotel_review"]/div/ul[2]/li/div/div[2]/div[2]/div[12]/ul/li[3]/a/i').click()
        time.sleep(3)
except Exception as e:
    raise e
finally:
    fp.close()

正在爬取第6页
正在爬取第6页第1条评论
正在爬取第6页第2条评论
正在爬取第6页第3条评论
正在爬取第6页第4条评论
正在爬取第6页第5条评论
正在爬取第6页第6条评论
正在爬取第6页第7条评论
正在爬取第6页第8条评论
正在爬取第6页第9条评论
正在爬取第6页第10条评论
正在爬取第7页
正在爬取第7页第1条评论
正在爬取第7页第2条评论
正在爬取第7页第3条评论
正在爬取第7页第4条评论
正在爬取第7页第5条评论
正在爬取第7页第6条评论
正在爬取第7页第7条评论
正在爬取第7页第8条评论
正在爬取第7页第9条评论
正在爬取第7页第10条评论
正在爬取第8页
正在爬取第8页第1条评论
正在爬取第8页第2条评论
正在爬取第8页第3条评论
正在爬取第8页第4条评论
正在爬取第8页第5条评论
正在爬取第8页第6条评论
正在爬取第8页第7条评论
正在爬取第8页第8条评论
正在爬取第8页第9条评论
正在爬取第8页第10条评论
正在爬取第9页
正在爬取第9页第1条评论
正在爬取第9页第2条评论
正在爬取第9页第3条评论
正在爬取第9页第4条评论
正在爬取第9页第5条评论
正在爬取第9页第6条评论
正在爬取第9页第7条评论
正在爬取第9页第8条评论
正在爬取第9页第9条评论
正在爬取第9页第10条评论
正在爬取第10页
正在爬取第10页第1条评论
正在爬取第10页第2条评论
正在爬取第10页第3条评论
正在爬取第10页第4条评论
正在爬取第10页第5条评论
正在爬取第10页第6条评论
正在爬取第10页第7条评论
正在爬取第10页第8条评论
正在爬取第10页第9条评论
正在爬取第10页第10条评论
正在爬取第11页
正在爬取第11页第1条评论
正在爬取第11页第2条评论
正在爬取第11页第3条评论
正在爬取第11页第4条评论
正在爬取第11页第5条评论
正在爬取第11页第6条评论
正在爬取第11页第7条评论
正在爬取第11页第8条评论
正在爬取第11页第9条评论
正在爬取第11页第10条评论
正在爬取第12页
正在爬取第12页第1条评论
正在爬取第12页第2条评论
正在爬取第12页第3条评论
正在爬取第12页第4条评论
正在爬取第12页第5条评论
正在爬取第12页第6条评论
正在爬取第12页第7条评论
正在爬取第12页第8条评论
正在爬取第12页第9条评论
正在爬取第12页第

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="ibu_hotel_review"]/div/ul[2]/li/div/div[2]/div[2]/div[12]/ul/li[3]/a/i"}
  (Session info: chrome=120.0.6099.130); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6B4A72142+3514994]
	(No symbol) [0x00007FF6B4690CE2]
	(No symbol) [0x00007FF6B45376AA]
	(No symbol) [0x00007FF6B4581860]
	(No symbol) [0x00007FF6B458197C]
	(No symbol) [0x00007FF6B45C4EE7]
	(No symbol) [0x00007FF6B45A602F]
	(No symbol) [0x00007FF6B45C28F6]
	(No symbol) [0x00007FF6B45A5D93]
	(No symbol) [0x00007FF6B4574BDC]
	(No symbol) [0x00007FF6B4575C64]
	GetHandleVerifier [0x00007FF6B4A9E16B+3695259]
	GetHandleVerifier [0x00007FF6B4AF6737+4057191]
	GetHandleVerifier [0x00007FF6B4AEE4E3+4023827]
	GetHandleVerifier [0x00007FF6B47C04F9+689705]
	(No symbol) [0x00007FF6B469C048]
	

In [52]:
import pandas as pd

cmts = pd.read_csv('comments.csv')
cmts.head()

,评分,评价,评论,日期,位置
0,5.0,超棒,服务：房间很不错，喜乐方言晓琳小会,2023年12月27日,上海
1,4.0,NaN,干净卫生，智能窗帘，智能马桶。服务态度很好，早餐不错，特别是咖啡，是我喜欢的味道。地理位置优...,2023年8月11日,广东
2,5.0,超棒,房间很不错喜乐方言离地铁口近位置好,2023年12月27日,NaN
3,5.0,超棒,房间服务好喜乐方言小琳小会位置好离地铁近,2023年12月27日,NaN
4,5.0,超棒,方言喜乐小麦小会服务热情满意,2024年1月1日,上海
